In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import sklearn
import pickle
import pandas as pd
import numpy as np

## Load the pikle files

In [2]:
# load the trained model
model = load_model('ann_model.h5')

# load the scaler
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

# load the one-hot encoder
with open('one_hot_encoder_geo.pkl', 'rb') as file:
    ohe_encoder = pickle.load(file)

# load label encoder 

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

In [3]:
# Example input data

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [14]:
# One hot encode Geography
geo_encoded = ohe_encoder.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_encoder.get_feature_names_out(['Geography']))

d:\GenAI With Langchain\NLP\End-To-End Project with ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [19]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


## Encode categorical features

In [20]:
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


## concatenate the encoded Geography with the input DataFrame

In [21]:
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


## scaling the input data

In [22]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

## prediction churn

In [23]:
prediction = model.predict(input_scaled)
prediction 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


array([[0.02672986]], dtype=float32)

In [24]:
prediction_prob = prediction[0][0] # fetching this 0.02672986 from this array([[0.02672986]], dtype=float32)

if prediction_prob > 0.5:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.
